In [1]:
!pip install -r requirements.txt 

Defaulting to user installation because normal site-packages is not writeable


search all url apartments and check to last page

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

def list_url_on_apartment(url_page: str):
    driver = webdriver.Chrome()
    driver.get(url=url_page)
    driver.implicitly_wait(10)
    
    src = driver.page_source
    soup = BeautifulSoup(src, "lxml")
    apartment_catalog = soup.find_all("div", class_ = "css-1sw7q4x")
    
    list_on_apartment = []
    for apartment in apartment_catalog:
        try:
            apartment_url = apartment.find("a").get("href")
            list_on_apartment.append(apartment_url)
        except AttributeError:
            driver.quit()
            return list_on_apartment

    driver.quit()
    return list_on_apartment

def is_last_page(url_page: str):
    driver = webdriver.Chrome()
    driver.get(url=url_page)
    driver.implicitly_wait(10)
    
    src = driver.page_source
    soup = BeautifulSoup(src, "lxml")
    if soup.find("a", {'data-testid': 'pagination-forward'}):
        return False
    else:
        return True
    

In [3]:

base_url = 'https://www.olx.ua/uk/nedvizhimost/kvartiry/'
page_number = 1
list_all_url_apartment = []

has_new_result = True
while True:
    url = f'{base_url}?page={page_number}'
    list_all_url_apartment.append(list_url_on_apartment(url))
    if is_last_page(url):
        break
    page_number += 1


search all information about apartment

In [4]:
import re

list_info = ["область", "Поверх", "Поверховість", "Загальна площа"]

def apartment_info(apartment_url: str):
    driver = webdriver.Chrome()
    driver.get(url=apartment_url)
    driver.implicitly_wait(10)
    
    src = driver.page_source
    soup = BeautifulSoup(src, "lxml")
    price_element = soup.find("h3", class_="css-12vqlj3")
    dict_info = {}
    for param_info in list_info:
        try:
            param = soup.find("p", text=lambda text: re.search(param_info, text))
            if param:
                param_text = param.text.split(":")[1].strip()
                dict_info[param_info] = param_text
            else:
                dict_info[param_info] = "None"
        except:
            dict_info[param_info] = "None"
    if price_element:
        price_text = price_element.text.strip()
        price = price_text.replace(' ', '')
        dict_info["ціна"] = price
    else:
        dict_info["ціна"] = "None"
    driver.quit()
    return dict_info


collection of all links

In [5]:
import pandas as pd

list_of_dicts = []
list_all_url_apartment = sum(list_all_url_apartment, [])

for apartment_url in list_all_url_apartment:
    apartment_info_dict = apartment_info(f"https://www.olx.ua{apartment_url}")
    list_of_dicts.append(apartment_info_dict)

df_sheet = pd.DataFrame(list_of_dicts)

KeyboardInterrupt: 

save data in google sheet

In [ ]:
from google.oauth2.service_account import Credentials
import gspread

scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = Credentials.from_service_account_file('PATH_YOUR_CREDENTIALS/auth_cred.json', scopes=scope)
sheet_id = 'WRITE_YOUR_SHEET_ID'

client = gspread.authorize(creds) 
sheet_place = client.open_by_key(sheet_id)

worksheet = sheet_place.get_worksheet(0)
worksheet.update([df_sheet.columns.values.tolist()] + df_sheet.values.tolist())